Import dependencies

In [1]:
import landmark_detector as ld
import os
import numpy as np
import json

Set variables

In [2]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
path = '../preprocessing/dataset/train/'

In [3]:
num_videos = 0

for word in words:
    i = 1
    video_path = path + word + '/0001.mp4'
    while os.path.exists(video_path):
        try:
            num_videos += 1
            i += 1
        except:
            break
        video_path = path + word + '/' + str(i).zfill(4) + '.mp4'

print('Number of videos:', num_videos)

Number of videos: 510


data:

videos
    frames
        hands
            landamrks
                x
                y
                z
    word

(number_of_samples, sequence_length, number_of_features).
(510,32,(2,21,3))


In [ ]:
select_words = ['deaf']

detector = ld.get_detector('../models/hand_landmarker.task')

training_X = []


for word in select_words:
    i = 1
    video_path = path + word + '/0001.mp4'
    while os.path.exists(video_path):
        try:
            landmarks = ld.get_landmarks(video_path, detector)
            i += 1
            print(np.shape(landmarks))
            training_X.append(landmarks)
        except:
            break
        video_path = path + word + '/' + str(i).zfill(4) + '.mp4'

print(training_X)

(3, 1, 21, 3)
(3, 1, 21, 3)
(4, 1, 21, 3)
(10, 1, 21, 3)
[[[[[0.53946852684021, 0.6340251564979553, 6.375985606155155e-08], [0.6187006831169128, 0.6095080971717834, 0.007846344262361526], [0.6553698778152466, 0.5701025128364563, 0.005429905839264393], [0.6786462068557739, 0.5424555540084839, -0.0048616621643304825], [0.6913325786590576, 0.520948052406311, -0.010716627351939678], [0.5820295810699463, 0.49746090173721313, 0.01675698161125183], [0.6566315293312073, 0.48003333806991577, -0.008544430136680603], [0.6705014109611511, 0.5055933594703674, -0.02611386962234974], [0.6602155566215515, 0.5261260867118835, -0.03210970759391785], [0.5747711062431335, 0.5034083127975464, 0.0010292239021509886], [0.6649046540260315, 0.4868820011615753, -0.01837221160531044], [0.6782974004745483, 0.5142509341239929, -0.02521318383514881], [0.6661949157714844, 0.5345711708068848, -0.025998743250966072], [0.5790598392486572, 0.518237829208374, -0.01792881265282631], [0.6713680028915405, 0.5088372826576233